In [1]:
import cython

In [2]:
%load_ext Cython
%load_ext autotime

In [3]:
import sys
sys.path.append('/Users/Pradap/Documents/Research/Python-Package/enrique/')
import magellan as mg
import re, string

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 444 ms


In [4]:
mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')


True

time: 128 ms


In [5]:
ob = mg.OverlapBlocker()

time: 1.79 ms


In [6]:
A = mg.load_dataset('table_A')
B = mg.load_dataset('table_B')

time: 12.5 ms


In [7]:
A

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


time: 15.1 ms


In [8]:
a_df = A.to_dataframe()
b_df = B.to_dataframe()

time: 9.1 ms


In [9]:
a_dict = {}
for k, r in a_df.iterrows():
    a_dict[k] = r
b_dict = {}
for k, r in b_df.iterrows():
    b_dict[k] = r

time: 14.2 ms


In [10]:
l_col_values_chopped = ob.process_table(a_df, 'address', None, True)
r_col_values_chopped = ob.process_table(b_df, 'address', None, True)

time: 3.26 ms


In [11]:
d = {}
def probe_inv_index(token, inv_index):
    return inv_index.get(token, None)
    

time: 2.1 ms


In [12]:
zipped_col_values = zip(l_col_values_chopped, range(0, len(l_col_values_chopped)))
appended_col_idx_values = [ob.append_index_values(v[0], v[1]) for v in zipped_col_values]
inv_idx = {}
sink = [ob.compute_inv_index(t, inv_idx) for c in appended_col_idx_values for t in c]

time: 3.23 ms


In [13]:
r = r_col_values_chopped[0]

time: 1.32 ms


In [32]:
zipped_r_col_values = zip(r_col_values_chopped, 
                          range(0, len(r_col_values_chopped)))

time: 1.2 ms


In [33]:
zipped_r_col_values

[(['108', 'clement', 'st', 'san', 'francisco'], 0),
 (['3131', 'webster', 'st', 'san', 'francisco'], 1),
 (['1652', 'stockton', 'st', 'san', 'francisco'], 2),
 (['108', 'south', 'park', 'san', 'francisco'], 3),
 (['170', 'post', 'st', 'apt', '4', 'san', 'francisco'], 4),
 (['133', 'clement', 'street', 'san', 'francisco'], 5)]

time: 4.67 ms


In [43]:
    from collections import Counter
    def probe_inv_index_for_a_token(token, inverted_index):
         return inverted_index.get(token, None)

    def probe_inv_index(ls, inverted_index):
        return [probe_inv_index_for_a_token(t, inverted_index) for t in ls]

    def get_freq_count( ls):
        p = list()
        dummy = [p.extend(k) for k in ls if k is not None]
        d = dict(Counter(p))
        return d

time: 4.75 ms


In [44]:
kk = [probe_inv_index(r[0], inv_idx) for r in zipped_r_col_values]

time: 1.04 ms


In [45]:
mm = [get_freq_count(k) for k in kk]

time: 1.02 ms


[{0: 3, 1: 3, 2: 3, 3: 3, 4: 2},
 {0: 3, 1: 3, 2: 5, 3: 3, 4: 2},
 {0: 3, 1: 5, 2: 3, 3: 3, 4: 2},
 {0: 2, 1: 2, 2: 2, 3: 2, 4: 2},
 {0: 3, 1: 3, 2: 3, 3: 3, 4: 3},
 {0: 2, 1: 2, 2: 2, 3: 2, 4: 3}]

time: 3.79 ms
